In [1]:

%load_ext autoreload
%autoreload 2


In [2]:
import numpy as np
import pickle
import uproot as ur
import awkward as ak
import matplotlib.pyplot as plt

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

#RUN BEFORE#

In [4]:
#RUN AFTER#

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import tensorflow.keras as keras
K = keras.backend

from util.Models import *
from util.Generators import *
from util.Plotting import *


In [5]:

data_path = '/fast_scratch_1/atlas_images/v01-45/'

cell_geo_path = data_path + 'cell_geo.root'

out_path = '/fast_scratch_1/jlerner/data/'


In [6]:

with ur.open(data_path + 'pi0/user.angerami.24559740.OutputStream._000232.root') as file:
    pi0_data = file['EventTree'].arrays(library='ak')
    
with ur.open(data_path + 'pipm/user.angerami.24559744.OutputStream._000232.root') as file:
    pipm_data = file['EventTree'].arrays(library='ak')
    
orig_pred = np.concatenate((ak.flatten(pipm_data['cluster_E']), 
                            ak.flatten(pi0_data['cluster_E']))).to_numpy()

orig_target = np.concatenate((ak.flatten(pipm_data['cluster_ENG_CALIB_TOT']), 
                              ak.flatten(pi0_data['cluster_ENG_CALIB_TOT']))).to_numpy()


In [7]:
norm = 'log' # 'log', 'std', or 'max'

if norm == 'log':
    scaler = None
elif norm == 'std':
    scaler = StandardScaler()
elif norm == 'max':
    scaler = MinMaxScaler()

if scaler is not None:
    sample = orig_target.reshape(-1, 1)
    scaler.fit(sample)

normalizer = (norm, scaler)

In [8]:
preprocess = False
train = True
save = True

In [9]:
if train:
    K.clear_session()
    model = GarNetModel()
    model.summary()

2023-06-13 13:51:57.003691: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2023-06-13 13:51:57.005138: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6915 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 4)]     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
garnet (GarNetStack)            (None, 16)           2976        input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 16)           272         garnet[0][0]                 

In [10]:

train_val_split = 0.8
batch_size = 64

pi0_list = [[data_path + f'pi0/user.angerami.24559740.OutputStream._000{i:03d}.root', 1] 
            for i in list(range(11, 113)) + list(range(116, 432))]
pipm_list = [[data_path + f'pipm/user.angerami.24559744.OutputStream._000{i:03d}.root', 0] 
             for i in list(range(11, 113)) + list(range(116, 432))]

np.random.shuffle(pi0_list)
np.random.shuffle(pipm_list)

train_start = 0
train_end = train_start + int(train_val_split*len(pi0_list))
val_start = train_end
val_end = len(pi0_list)
train_file_list = (pi0_list[train_start:train_end], pipm_list[train_start:train_end])
val_file_list = (pi0_list[val_start:val_end], pipm_list[val_start:val_end])

test_file_list = ([[data_path + f'pi0/user.angerami.24559740.OutputStream._000{i:03d}.root', 1] for i in range(432, 464)],
                  [[data_path + f'pipm/user.angerami.24559744.OutputStream._000{i:03d}.root', 0] for i in range(432, 464)])


In [11]:

train_generator = garnetDataGenerator(train_file_list, 
                                      cell_geo_path, 
                                      batch_size,
                                      normalizer=normalizer,
                                      name='garnet_' + normalizer[0],
                                      labeled=True, 
                                      preprocess=preprocess, 
                                      output_dir=out_path + 'train/')

if preprocess: cell_geo_path = train_generator.geo_dict

validation_generator = garnetDataGenerator(val_file_list, 
                                           cell_geo_path,
                                           int(batch_size*(1 - train_val_split)/train_val_split),
                                           normalizer=normalizer,
                                           name='garnet_' + normalizer[0],
                                           labeled=True, 
                                           preprocess=preprocess, 
                                           output_dir=out_path + 'val/')

test_generator = garnetDataGenerator(test_file_list,
                                     cell_geo_path,
                                     batch_size=20000,
                                     normalizer=normalizer,
                                     name='garnet_' + normalizer[0],
                                     labeled=True,
                                     preprocess=preprocess,
                                     output_dir=out_path + 'test/')


In [ ]:
if train:
    
    #def scheduler(epoch, lr):
        #if epoch < 20:
            #return lr
        #else:
            #return lr*tf.math.exp(-0.10)
    #lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

    #def regression_loss(y_true, y_pred):
        #return K.mean(K.square((y_true - y_pred) / (y_true + K.epsilon())), axis=-1)
    regression_loss = keras.losses.MeanSquaredError()
    classification_loss = keras.losses.BinaryCrossentropy()
    
    optimizer=keras.optimizers.Adam(learning_rate=0.0005)
    losses = {'regression': regression_loss, 'classification': classification_loss}
    loss_weights = {'regression': 0.90, 'classification': 0.10}
    
    def loss_fcn(y_true, y_pred):
        return loss_weights['regression']*losses['regression'](y_true[:,2:3], y_pred[:,2:3]) + \
               loss_weights['classification']*losses['classification'](y_true[:,0], y_pred[:,0])
        
    model.compile(optimizer=optimizer, loss=losses, loss_weights=loss_weights, metrics=['accuracy'])
    
    callbacks = [PrinterCallback(), 
                 keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=5, verbose=1),
                 keras.callbacks.EarlyStopping(verbose=1, patience=10)]
    
    history = model.fit(train_generator.generator(), 
                        validation_data=validation_generator.generator(),
                        steps_per_epoch=3200,
                        validation_steps=800,
                        shuffle=True,
                        epochs=100,
                        callbacks=[PrinterCallback()],
                        verbose=0)

2023-06-13 13:52:02.172559: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 0/100: [==================================================] 3200/3200
72s - loss: 1.2408 - val loss: 0.7041
Epoch 1/100: [==================================================] 3200/3200
62s - loss: 0.6115 - val loss: 0.6114
Epoch 2/100: [==================================================] 3200/3200
58s - loss: 0.5391 - val loss: 0.5220
Epoch 3/100: [==================================================] 3200/3200
62s - loss: 0.5040 - val loss: 0.4719
Epoch 4/100: [==================================================] 3200/3200
65s - loss: 0.4810 - val loss: 0.4429
Epoch 5/100: [==================================================] 3200/3200
63s - loss: 0.4639 - val loss: 0.4454
Epoch 6/100: [==================================================] 3200/3200
64s - loss: 0.4579 - val loss: 0.4873
Epoch 7/100: [==================================================] 3200/3200
64s - loss: 0.4442 - val loss: 0.4528
Epoch 8/100: [==================================================] 3200/3200
64s - loss: 

In [ ]:
if save:
    model.save(out_path + 'models/GarNet_' + normalizer[0])

    with open(out_path + 'models/GarNet_' + normalizer[0] + '/history.pickle', 'wb') as handle:
        pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    with open(out_path + 'models/GarNet_' + normalizer[0] + '/scaler.pickle', 'wb') as handle:
        pickle.dump(scaler, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
K.clear_session()
model = tf.keras.models.load_model(out_path + 'models/GarNet_' + normalizer[0], 
                                   custom_objects={"GarNetModel": GarNetModel}, compile=False)

with open(out_path + 'models/GarNet_' + normalizer[0] + '/history.pickle', "rb") as file:
    history = pickle.load(file)
    
with open(out_path + 'models/GarNet_' + normalizer[0] + '/scaler.pickle', "rb") as file:
    scaler = pickle.load(file)

In [ ]:
x, y = next(test_generator.generator())

In [ ]:
loss_curve = Plotter(training, history=history, metrics=['loss', 'classification_loss', 'regression_loss'], scale='log')
loss_curve.show()

In [ ]:
accuracy_curve = Plotter(training, history=history, metrics=['regression_accuracy'])
accuracy_curve.show()

In [ ]:
ROC = Plotter(roc, preds=[model.predict(x)[0][:,0]], targets=[y['classification'][:,0]], labels=[''])
ROC.show()

In [ ]:
if normalizer[0] == 'log':
    fit_pred = np.exp(model.predict(x)[-1]).reshape(-1,)
    fit_target = np.exp(y['regression']).reshape(-1,)
else:
    scaler = normalizer[1]
    fit_pred = scaler.inverse_transform(model.predict(x)[-1]).reshape(-1,)
    fit_target = scaler.inverse_transform(np.reshape(y['regression'], (-1, 1))).reshape(-1,)

reg = Plotter(regResponse, 
              pred=fit_pred, 
              target=fit_target,
              stat=['median'])
reg.show()

In [ ]:
reg = Plotter(regResponseOverlay,
              preds=[orig_pred, fit_pred],
              targets=[orig_target, fit_target],
              labels=['Pre-Fit', 'Post-Fit'],
              stat=['mean', 'stdmean'])